<a href="https://colab.research.google.com/github/Pablo-dev-uni/CartPole-v0-/blob/master/Cart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import gym
import random
import numpy as np
import tflearn
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from statistics import median, mean
from collections import Counter

LR = 1e-3
env = gym.make("CartPole-v0")
env.reset()
goal_steps = 500
score_requirement = 50
initial_games = 50000

def some_random_games_first():
    # Each of these is its own game.
    for episode in range(5):
        env.reset()
        # this is each frame, up to 200...but we wont make it that far.
        for t in range(200):
            # This will display the environment
            # Only display if you really want to see it.
            # Takes much longer to display it.
            
            
            # This will just create a sample action in any environment.
            # In this environment, the action can be 0 or 1, which is left or right
            action = env.action_space.sample()
            
            # this executes the environment with an action, 
            # and returns the observation of the environment, 
            # the reward, if the env is over, and other info.
            observation, reward, done, info = env.step(action)
            if done:
              break
            return done
def initial_population():
    # [OBS, MOVES]
    training_data = []
    # all scores:
    scores = []
    # just the scores that met our threshold:
    accepted_scores = []
    # iterate through however many games we want:
    for _ in range(initial_games):
        score = 0
        # moves specifically from this environment:
        game_memory = []
        # previous observation that we saw
        prev_observation = []
        # for each frame in 200
        for _ in range(goal_steps):
            # choose random action (0 or 1)
            action = random.randrange(0,2)
            # do it!
            observation, reward, done, info = env.step(action)
            
            # notice that the observation is returned FROM the action
            # so we'll store the previous observation here, pairing
            # the prev observation to the action we'll take.
            if len(prev_observation) > 0 :
                game_memory.append([prev_observation, action])
            prev_observation = observation
            score+=reward
            if done: break

        # IF our score is higher than our threshold, we'd like to save
        # every move we made
        # NOTE the reinforcement methodology here. 
        # all we're doing is reinforcing the score, we're not trying 
        # to influence the machine in any way as to HOW that score is 
        # reached.
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                # convert to one-hot (this is the output layer for our neural network)
                if data[1] == 1:
                    output = [0,1]
                elif data[1] == 0:
                    output = [1,0]
                    
                # saving our training data
                training_data.append([data[0], output])

        # reset env to play again
        env.reset()
        # save overall scores
        scores.append(score)
    
    # just in case you wanted to reference later
    training_data_save = np.array(training_data)
    np.save('saved.npy',training_data_save)
    
    # some stats here, to further illustrate the neural network magic!
    print('Average accepted score:',mean(accepted_scores))
    print('Median score for accepted scores:',median(accepted_scores))
    print(Counter(accepted_scores))
    
    return training_data

  
 
           

W0826 07:49:10.865820 140533649205120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/helpers/summarizer.py:9: The name tf.summary.merge is deprecated. Please use tf.compat.v1.summary.merge instead.

W0826 07:49:10.870924 140533649205120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/helpers/trainer.py:25: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.

W0826 07:49:10.888292 140533649205120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/collections.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0826 07:49:10.897432 140533649205120 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tflearn/config.py:123: The name tf.get_collection is deprecated. Please use tf.compat.v1.get_collection instead.

W0826 07:49:10.912980 140533649205120 deprecation_wrapper.py:119] From /usr/local/lib/

In [0]:
def neural_network_model(input_size):
    model = Sequential()
    model.add(Dense(4,activation = 'relu',input_shape = (4,)))
    model.add(Dense(16,activation = 'relu'))
    model.add(Dense(4,activation = 'relu'))
    model.add(Dense(2,activation = 'softmax'))
    
    model.compile(optimizer = 'adam',loss = 'categorical_crossentropy')

    return model

In [0]:
def train_model(training_data, model=False):
    Xt = np.array([i[0] for i in training_data]).reshape(-1,len(training_data[0][0]))
    yt = np.array([i[1] for i in training_data])

    if not model:
        model = neural_network_model(input_size = (len(Xt[1])))
    
    model.fit(Xt, yt, epochs=10, batch_size=50)
    return model

In [15]:
training_data = initial_population()


Average accepted score: 61.17097862767154
Median score for accepted scores: 57.0
Counter({50.0: 147, 51.0: 141, 53.0: 118, 52.0: 110, 54.0: 106, 55.0: 95, 56.0: 91, 58.0: 83, 57.0: 83, 60.0: 68, 63.0: 67, 59.0: 67, 62.0: 51, 66.0: 46, 61.0: 42, 65.0: 41, 64.0: 38, 67.0: 31, 69.0: 27, 70.0: 24, 68.0: 23, 75.0: 20, 73.0: 19, 71.0: 18, 78.0: 18, 80.0: 14, 84.0: 13, 72.0: 13, 74.0: 12, 77.0: 11, 76.0: 11, 79.0: 10, 82.0: 9, 85.0: 9, 83.0: 8, 81.0: 8, 99.0: 7, 88.0: 6, 87.0: 6, 86.0: 6, 91.0: 4, 89.0: 4, 100.0: 4, 98.0: 4, 93.0: 4, 120.0: 3, 102.0: 3, 103.0: 3, 95.0: 3, 92.0: 2, 110.0: 2, 96.0: 2, 97.0: 2, 108.0: 2, 114.0: 2, 106.0: 2, 94.0: 2, 90.0: 2, 105.0: 2, 109.0: 1, 116.0: 1, 128.0: 1, 111.0: 1, 119.0: 1, 131.0: 1, 101.0: 1, 115.0: 1, 112.0: 1})


In [16]:
model = train_model(training_data)

Epoch 1/10
106984/106984 [==============================] - 3s 26us/step - loss: 0.6705
Epoch 2/10
106984/106984 [==============================] - 3s 24us/step - loss: 0.6619
Epoch 3/10
106984/106984 [==============================] - 2s 23us/step - loss: 0.6610
Epoch 4/10
106984/106984 [==============================] - 2s 23us/step - loss: 0.6604
Epoch 5/10
106984/106984 [==============================] - 2s 23us/step - loss: 0.6603
Epoch 6/10
106984/106984 [==============================] - 3s 24us/step - loss: 0.6600
Epoch 7/10
106984/106984 [==============================] - 2s 23us/step - loss: 0.6600
Epoch 8/10
106984/106984 [==============================] - 2s 23us/step - loss: 0.6599
Epoch 9/10
106984/106984 [==============================] - 2s 23us/step - loss: 0.6597
Epoch 10/10
106984/106984 [==============================] - 2s 22us/step - loss: 0.6596


In [22]:
scores = []
choices = []
for each_game in range(10):
    score = 0
    game_memory = []
    prev_obs = []
    env.reset()
    for _ in range(goal_steps):
      if len(prev_obs)==0:
        action = random.randrange(0,2)
      else:
        action = np.argmax(model.predict(prev_obs.reshape(1,len(prev_obs)))[0])

      choices.append(action)
                
      new_observation, reward, done, info = env.step(action)
      prev_obs = new_observation
      game_memory.append([new_observation, action])
      score+=reward
      if done: break

    scores.append(score)

print('Average Score:',sum(scores)/len(scores))
print('choice 1:{}  choice 0:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices)))
print(score_requirement)

Average Score: 192.0
choice 1:0.5010416666666667  choice 0:0.49895833333333334
50
